## Parte 1: Análisis Básico con NumPy

En esta primera parte del proyecto, los estudiantes realizarán un análisis preliminar del dataset utilizando NumPy. El objetivo es familiarizarse con los datos y realizar operaciones básicas de manipulación y análisis.

### Instrucciones:
  #### Configuración Inicial del Proyecto:
  - Crea un repositorio en GitHub para tu proyecto.
  - Configura dos ramas en tu repositorio: main y development.
  - Agrega un archivo README.md con una descripción del proyecto, instrucciones de instalación y uso.

#### Carga y Preprocesamiento de Datos:
  - Carga los datos del archivo CSV utilizando NumPy.
  - Realiza un preprocesamiento básico para asegurarte de que los datos estén limpios y listos para su análisis.

#### Exploración de Datos:
  - Calcula el total de ventas por categoría de producto.
  - Calcula el promedio de ventas diarias por categoría de producto.
  - Identifica las categorías de productos con mayores y menores ventas.

#### Manipulación de Datos:
  - Filtra los datos para mostrar solo las ventas de una categoría de producto específica.
  - Realiza operaciones de suma, resta, multiplicación y división en los datos para obtener estadísticas adicionales.

In [62]:
# carga de datos && comprobación de datos nulos && Limpieza de datos
import numpy as np
from collections import defaultdict
from datetime import datetime

def cargar_datos(ruta_archivo):
    # Cargar el CSV como un array estructurado, con nombres de columnas y dtype automático
    datos = np.genfromtxt(ruta_archivo, delimiter=',', skip_header=0, names=True, dtype=None, encoding='utf-8')
    print(datos.dtype.names)
    # Verificar y reemplazar valores nulos en las columnas numéricas
    columnas_numericas = ['Transaction_ID', 'Age', 'Quantity', 'Price_per_Unit', 'Total_Amount']
    for columna in columnas_numericas:
        # Detectar si hay valores nulos en las columnas numéricas
        if np.isnan(datos[columna]).any():
            print(f'Tiene valores nulos en la columna {columna}... Reemplazando valores nulos con 0')
            datos[columna] = np.nan_to_num(datos[columna], nan=0)
    
    return datos

if __name__ == "__main__":
    ruta_archivo = '../data/retail_sales_dataset.csv'
    datos = cargar_datos(ruta_archivo)

('Transaction_ID', 'Date', 'Customer_ID', 'Gender', 'Age', 'Product_Category', 'Quantity', 'Price_per_Unit', 'Total_Amount')


In [63]:
# Exploracion de datos

def calcular_ventas_por_categoria(datos):
    # Calculo de total de ventas por categoria
    categorias = datos['Product_Category']  # Extraer las columnas relevantes: categoría del producto (índice 5) y monto total (índice 8)
    ventas_totales = datos['Total_Amount'] # Columna de montos totales, convertidos a float
    # Crear un diccionario para almacenar las ventas totales por categoría
    ventas_por_categoria = {}
    # Recorrer los datos para agrupar y sumar por categoría
    for categoria, venta in zip(categorias, ventas_totales):
        if categoria in ventas_por_categoria:
            ventas_por_categoria[categoria] += venta
        else:
            ventas_por_categoria[categoria] = venta
    
    # Convertir el diccionario en una lista de tuplas [(categoria, total_ventas), ...]
    lista_ventas_categoria = [(categoria, total) for categoria, total in ventas_por_categoria.items()]
    return lista_ventas_categoria

def calcular_categorias_mas_menos_ventas(ventas_categoria):
    ventas_categoria = sorted(ventas_categoria, key=lambda x: x[1], reverse=True)
    mas_ventas = ventas_categoria[0]
    menos_ventas = ventas_categoria[-1]
    return mas_ventas, menos_ventas

def calcular_promedio_ventas_diarias_por_categoria(datos):
    # Diccionarios para acumular las ventas totales por categoría y los días de ventas
    ventas_por_categoria = defaultdict(float)
    dias_por_categoria = defaultdict(set)  # Usamos set para evitar contar días duplicados
    # Recorrer el dataset
    for fila in datos:
        categoria = fila['Product_Category']
        fecha = fila['Date']
        ventas = fila['Total_Amount']
        # Convertir la fecha a objeto datetime para garantizar que sea comparable
        fecha_convertida = datetime.strptime(fecha, '%Y-%m-%d')
        # Acumular ventas por categoría
        ventas_por_categoria[categoria] += ventas
        # Agregar la fecha al conjunto de días por categoría (evita contar días repetidos)
        dias_por_categoria[categoria].add(fecha_convertida)
    # Diccionario para almacenar el promedio de ventas diarias por categoría
    promedio_ventas_diarias = {}
    # Calcular el promedio de ventas diarias por categoría
    for categoria in ventas_por_categoria:
        total_ventas = ventas_por_categoria[categoria]
        total_dias = len(dias_por_categoria[categoria])
        if total_dias > 0:
            promedio_ventas_diarias[categoria] = total_ventas / total_dias
        else:
            promedio_ventas_diarias[categoria] = 0  # Evitar división por cero
    return promedio_ventas_diarias

# Calcular el total de ventas por categoría
ventas_categoria = calcular_ventas_por_categoria(datos)
print("Ventas totales por categoría:")
for categoria, total in ventas_categoria:
    print(f"{categoria}: {total}")
print('-----------------------------------------', end='\n')

# Calcular categoria con mayor y menor venta
mas_ventas, menos_ventas =calcular_categorias_mas_menos_ventas(ventas_categoria)
print(f'Categoria con más ventas: {mas_ventas[0]} => {mas_ventas[1]}')
print(f'Categoria con menos ventas: {menos_ventas[0]} => {menos_ventas[1]}')
print('-----------------------------------------', end='\n')

# Calcular el promedio de ventas diarias por categoría
promedio_ventas_categoria = calcular_promedio_ventas_diarias_por_categoria(datos)
print("Promedio de ventas diarias por categoría:")
for categoria, promedio in promedio_ventas_categoria.items():
    print(f"{categoria}: {promedio:.2f}")
print('-----------------------------------------', end='\n')


Ventas totales por categoría:
Beauty: 143515
Clothing: 155580
Electronics: 156905
-----------------------------------------
Categoria con más ventas: Electronics => 156905
Categoria con menos ventas: Beauty => 143515
-----------------------------------------
Promedio de ventas diarias por categoría:
Beauty: 703.50
Clothing: 670.60
Electronics: 716.46
-----------------------------------------


In [79]:
# Filtra los datos para mostrar solo las ventas de una categoría de producto específica.
# Realiza operaciones de suma, resta, multiplicación y división en los datos para obtener estadísticas adicionales.
def mostrar_ventas_filtro_categoria(datos, categoria):
    ventas_totales = 0
    for row in datos:
        if categoria in row['Product_Category']:
            ventas_totales += row['Total_Amount']
    return ventas_totales

categoria = 'Electronics'
ventas_totales = mostrar_ventas_filtro_categoria(datos, categoria)
print(f'Ventas totales de la categoria: {categoria} => {ventas_totales}') 

Ventas totales de la categoria: Electronics => 156905
